# Model Free Policy Evaluation
## Example 1: Random Walk
### TODO:
- [x] Implement Monte-Carlo
- [x] Implement Temporal Difference TD(0)
    - [x] Plot RMS err vs Walks per ep across different `alpha` for both MC & TD(0)
    - [x] Contrast with those in slides, idk why but they are hella different
- [ ] Implement n-step TD 
    - [ ] Plot RMS vs `alpha` for different `n` for both online and offline setting
    - [ ] Contrast with those in slides
- [ ] Implement TD(`lambda`)
    - [ ] Plot RMS vs `alpha` for different `lambda` for offline setting 
    - [ ] Contrast with those in slides

Next: Blackjack

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

def get_reward(cfg, state):
    if state == -1:
        return 0
    elif state == cfg['road_len']:
        return 1
    else:
        return 0


def get_next_pos(cfg, curr_pos, values):
    def rand_next(curr_pos):
        if np.random.rand() < 0.5:
            curr_pos -= 1
        else:
            curr_pos += 1

        return curr_pos
    
    def greedy_next(curr_pos, values):
        if curr_pos - 1 < 0:
            curr_pos += 1
        elif curr_pos + 1 >= cfg['road_len']: 
            curr_pos -= 1
        else:
            if values[curr_pos-1] < values[curr_pos+1]:
                curr_pos += 1
            elif values[curr_pos-1] > values[curr_pos+1]:
                curr_pos -= 1
            else:
                curr_pos = rand_next(curr_pos)

        return curr_pos
    
    if cfg['eval_policy'] == 'rand':
        curr_pos = rand_next(curr_pos)
    elif cfg['eval_policy'] == 'greedy':
        curr_pos = greedy_next(curr_pos, values)
    elif cfg['eval_policy'] == 'eps-greedy':
        if np.random.rand() < cfg['eps-greedy/epsilon']:
            curr_pos = rand_next(curr_pos)
        else:
            curr_pos = greedy_next(curr_pos, values)

    return curr_pos

In [2]:
def solve_mdp(
        cfg,
        optimal_policy=True,
        optimality_threshold=1e-3):
    """
    Uses random policy
    """

    road_len = cfg['road_len']
    gamma = cfg['gamma']
    terminal_states = cfg['terminal_states']

    values = np.full(road_len, 0.0)
    policy = 0.5 * np.ones((road_len, 2))

    for _ in range(500):
        new_values = np.zeros_like(values)

        # Value Update
        for i in range(road_len):
            for action, i_new in enumerate([i-1, i+1]):
                curr_reward = get_reward(cfg, i_new)

                returns = 0.0 if i_new in terminal_states else values[i_new]
                
                new_values[i] += policy[i, action] * (curr_reward + gamma * returns)


        if (new_values - values < optimality_threshold).all():
            break

        values = new_values
        if optimal_policy is False:
            continue

        # Policy Update
        new_policy = np.zeros_like(policy)

        for i in range(road_len):
            next_values = np.zeros(2)
            for action, i_new in enumerate([i-1, i+1]):
                if i_new in terminal_states:
                    next_values[action] = -np.inf
                    continue
                
                new_values[action] = values[i_new]

            best_actions = np.where(next_values == next_values.max())[0]
            new_policy[i, best_actions] = 1
            new_policy[i] /= new_policy[i].sum()

    return values

In [3]:
def monte_carlo_eval(cfg, values):
    pos_rewards = []

    # Init episode
    if cfg['start_pos'] == 'mid':
        curr_pos = cfg['road_len'] // 2

    # Running/sampling episode
    for step in range(cfg['max_episode_len']):
        prev_pos = curr_pos
        
        curr_pos = get_next_pos(cfg, curr_pos, values)

        pos_rewards.append(
            (prev_pos, get_reward(cfg, curr_pos))
        )

        if curr_pos in cfg['terminal_states']:
            break

    # Updating values
    new_values = values.copy()
    returns = 0.0
    for pos, reward in reversed(pos_rewards):
        returns = reward + cfg['gamma'] * returns

        new_values[pos] += cfg['alpha'] * (returns - new_values[pos])

    return new_values

In [4]:
def temporal_difference_eval(cfg, values, lamda=0, n=1):
    # Init episode
    if cfg['start_pos'] == 'mid':
        curr_pos = cfg['road_len'] // 2

    # Running/sampling episode
    for step in range(cfg['max_episode_len']):
        prev_pos = curr_pos
        
        curr_pos = get_next_pos(cfg, curr_pos, values)

        reward = get_reward(cfg, curr_pos)
        
        expt_reward = reward 
        if curr_pos >= 0 and curr_pos < cfg['road_len']:
            expt_reward += cfg['gamma'] * values[curr_pos]

        td_err = expt_reward - values[prev_pos]

        values[prev_pos] += cfg['alpha'] * td_err

        if curr_pos in cfg['terminal_states']:
            break

    return values

In [5]:
def rms(values, gt):
    diff = values - gt
    return np.sqrt(np.mean(diff**2))

def plot_values(values_history, gt, title=''):
    ar = np.arange(gt.shape[0])

    for episode, values in values_history:
        plt.plot(ar, values, label=f'ep = {episode}')
    
    plt.plot(ar, gt, label='gt', color='black')

    plt.legend()
    plt.title(title)
    plt.show()

def plot_rms(rms_history, title=''):
    ar = np.arange(len(rms_history))

    plt.plot(ar, rms_history, color='black')
    plt.title(title)
    plt.show()

In [6]:
def train(cfg):
    num_episodes = cfg['num_episodes']
    
    values = np.full(cfg['road_len'], 0.5)

    gt = solve_mdp(cfg, optimal_policy=False)

    values_history = []
    rms_history = []
    for episode in range(num_episodes + 1):
        rms_history.append(rms(values, gt))

        if cfg['eval_policy'] == 'eps-greedy':
            x = (episode+1) / num_episodes
            cfg['eps-greedy/epsilon'] = 1 - np.log(x)
        
        if cfg['evaluator'] == 'mc':
            evaluator = monte_carlo_eval
        elif cfg['evaluator'] == 'td_0':
            evaluator = temporal_difference_eval

        values = evaluator(cfg, values)

        if episode % (num_episodes // 2) == 0:
            values_history.append((episode, values.copy()))

    title1 = '{} Value Function | alpha = {} | gamma = {}'.format(
        cfg['evaluator'], cfg['alpha'], cfg['gamma'])
    title2 = '{} RMS Plot | alpha = {} | gamma = {}'.format(
        cfg['evaluator'], cfg['alpha'], cfg['gamma'])
    
    # plot_values(values_history, gt, title1)
    # plot_rms(rms_history, title2)

    return rms_history

In [7]:
import matplotlib.cm as cm

def plot_alpha_rms_history(cfg, alpha_rms_historys, title=''):
    x = np.arange(cfg['num_episodes'] + 1)
    colormap = cm.get_cmap('viridis')

    for alpha, rms_historys in alpha_rms_historys.items():
        color = colormap(alpha * 5)
        y_min = np.min(rms_historys, axis=0)
        y_max = np.max(rms_historys, axis=0)
        plt.fill_between(x, y_min, y_max, color=color, alpha=0.25)

        mean_rms_history = np.mean(rms_historys, axis=0)
        plt.plot(x, mean_rms_history, label=f'alpha = {alpha}', color=color)
    
    plt.title(title)
    plt.legend()
    plt.xlabel('Episode')
    plt.ylabel('RMS Error')
    plt.show()

In [8]:
road_len = 10

cfg = {
    'road_len': road_len,

    'evaluator': 'td_0',  # ['mc', 'td_0', 'td_l']
    'num_episodes': 1000,
    'max_episode_len': 200,
    'start_pos': 'mid',

    # 'alpha': 0.01,
    'gamma': 0.99,
    'eval_policy': 'rand',  # ['rand', 'greedy', 'eps-greedy']    

    'terminal_states': [-1, road_len]
}

if False:
    alpha_rms_historys = defaultdict(list)

    itrs = 100
    for k in range(1, itrs+1):
        # for alpha in [0.01, 0.05, 0.10]:  # MC
        for alpha in [0.05, 0.1, 0.15]:  # TD
            cfg['alpha'] = alpha

            alpha_rms_historys[alpha].append(train(cfg))

    title = '{}: RMS averaged across {} runs | gamma = {}'.format(
        cfg['evaluator'].upper(),
        itrs,
        cfg['gamma']
    )
    plot_alpha_rms_history(cfg, alpha_rms_historys, title=title)

In [9]:
import matplotlib.cm as cm

def plot_alpha_rms(cfg, alpha_rms, title=''):
    x = alpha_rms.keys()
    colormap = cm.get_cmap('viridis')

    y_min, y_max, y_mean = [], [], []
    for alpha, rms in alpha_rms.items():
        y_min.append(np.min(rms))
        y_max.append(np.max(rms))
        y_mean.append(np.mean(rms))
    
    color = 'purple'
    plt.fill_between(x, y_min, y_max, color=color, alpha=0.5)
    plt.plot(x, y_mean, color=color)

    plt.title(title)
    plt.xlabel('alpha')
    plt.ylabel('RMS Error')
    plt.xscale('log')
    plt.show()

In [10]:
road_len = 10

cfg = {
    'road_len': road_len,

    'evaluator': 'mc',  # ['mc', 'td_0', 'td_l']
    'num_episodes': 250,
    'max_episode_len': 200,
    'start_pos': 'mid',

    # 'alpha': 0.01,
    'gamma': 0.9,
    'eval_policy': 'rand',  # ['rand', 'greedy', 'eps-greedy']    

    'terminal_states': [-1, road_len]
}

def err_alpha_plot(cfg):
    alpha_rms = defaultdict(list)

    itrs = 100
    for k in range(1, itrs+1):
        for alpha in np.logspace(-3, 0, 100):
            alpha -= 1e-3
            cfg['alpha'] = alpha

            alpha_rms[alpha].append(train(cfg)[-1])

    title = '{} For {} Episodes | gamma = {}'.format(
        cfg['evaluator'].upper(),
        cfg['num_episodes'],
        cfg['gamma']
    )

    plot_alpha_rms(cfg, alpha_rms, title=title)

cfg['gamma'] = 0.99
err_alpha_plot(cfg)

cfg['evaluator'] = 'td_0'
cfg['gamma'] = 0.9
err_alpha_plot(cfg)

cfg['evaluator'] = 'td_0'
cfg['gamma'] = 0.99
err_alpha_plot(cfg)

KeyboardInterrupt: 

In [3]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=1)

class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __eq__(self, other):
        if isinstance(other, tuple):
            return (self.x == other[0]) and (self.y == other[1])
        
        return (self.x == other.x) and (self.y == other.y)

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)

    def __truediv__(self, val):
        return Vector(self.x / val, self.y / val)
    
    def __str__(self):
        return f"({self.x}, {self.y})"
    
    def __int__(self):
        return (self.x, self.y)
    
    def __iter__(self):
        return iter((self.x, self.y))
    
    def copy(self):
        return Vector(self.x, self.y)

    def clip_update(self, new_vec, bounds):
        if new_vec.x < 0:
            self.x = 0
        elif new_vec.x >= bounds.x:
            self.x = bounds.x - 1
        else:
            self.x = new_vec.x

        if new_vec.y < 0:
            self.y = 0
        elif new_vec.y >= bounds.y:
            self.y = bounds.y - 1
        else:
            self.y = new_vec.y

In [48]:
class GridWorld:
    def __init__(self):
        self.shape = Vector(4,4)
        self.spawn_obstacle_p = 0.0

        np.random.seed(42)  # DO NOT change this
        self.occupancy_map = (np.random.rand(*self.shape) < self.spawn_obstacle_p).astype(int)
        
        self.start_pos = Vector(0,0)  # top left is (0, 0)
        self.goal_pos = Vector(self.shape.x-1, self.shape.y-1) 
        
        self.occupancy_map[*self.start_pos] = 0
        self.occupancy_map[*self.goal_pos] = 1

        self.actions = {
            0: Vector(0, -1), # N
            1: Vector(1, 0),  # E
            2: Vector(-1, 0), # W
            3: Vector(0, 1),  # S
        }

    def get_reward(self, agent_pos):
        if agent_pos == self.goal_pos:
            return 0

        if self.occupancy_map[tuple(agent_pos)] == 1:
            return -10

        return -1

class Agent:
    def __init__(self, world):
        self.world = world

        self.pos = self.world.start_pos
        self.actions = self.world.actions

        self.V = np.random.rand(*self.world.shape)
        self.V[*self.world.goal_pos] = 0
        self.pi = np.full((*self.world.shape, len(self.actions)), 1/len(self.actions))

        # self.Q = np.full((*self.world.shape, len(self.actions)), 1/len(self.actions))
    
    def init(self):
        self.pos = self.world.start_pos

In [41]:
class Trainer:
    def __init__(self, cfg):
        self.world = cfg['world']
        self.agent = Agent(cfg['world'])
        self.num_actions = len(self.agent.actions)

        self.max_episodes = cfg['max_episodes']
        self.lifespan = cfg['lifespan']

        self.gamma = cfg['gamma']

        self.mapping_dict = {
            0: '^',
            1: '>',
            2: '<',
            3: 'v'
        }

    def policy_iteration(self, theta=1e-3):
        """
        theta: threshold for stability of self.agent.V
        """
        
        def subroutine():
            # policy_evaluation
            while True:
                error = 0

                # looping over all states
                for y in range(self.world.shape.y):
                    for x in range(self.world.shape.x):
                        if self.world.occupancy_map[x, y] == 1:
                            self.agent.V[x, y] = self.world.get_reward((x, y))
                            continue

                        value = self.agent.V[x, y]
                        new_value = 0

                        for pi_i in range(self.num_actions):
                            new_pos = Vector(x, y) # placeholder name
                            new_pos.clip_update(new_pos + self.agent.actions[pi_i], self.world.shape)
                            
                            reward = self.world.get_reward(new_pos)
                            returns = self.agent.V[*new_pos]

                            new_value += self.agent.pi[x, y, pi_i] * (reward + self.gamma * returns)

                        error = max(error, np.abs(value - new_value))
                        self.agent.V[x, y] = new_value

                # Check exit cond for policy evaluation
                print(f'Error = {error}')
                if error < theta:
                    break
        
            # policy_improvement
            is_stable = True

            # looping over all states
            for y in range(self.world.shape.y):
                for x in range(self.world.shape.x):
                    if self.world.occupancy_map[x, y] == 1:
                        continue

                    old_action = np.argmax(self.agent.pi[x, y])
                    new_action_values = np.zeros(self.num_actions)

                    for pi_i in range(self.num_actions):
                        new_pos = Vector(x, y) # placeholder name
                        new_pos.clip_update(new_pos + self.agent.actions[pi_i], self.world.shape)

                        reward = self.world.get_reward(new_pos)
                        returns = self.agent.V[*new_pos]

                        new_action_values[pi_i] = self.agent.pi[x, y, pi_i] * (reward + self.gamma * returns)

                    best_pi = np.argmax(new_action_values)
                    self.agent.pi[x, y] = 0
                    self.agent.pi[x, y, best_pi] = 1


                    if old_action != best_pi:
                        is_stable = False

            print(f'is_stable = {is_stable}')
            # if is_stable: return
            # subroutine()


        for _ in range(10):
            subroutine()

        best_actions = np.argmax(self.agent.pi, axis=-1)
        vectorized_mapping = np.vectorize(self.mapping_dict.get)
        print(vectorized_mapping(best_actions).T)


In [52]:
cfg = {
    'world': GridWorld(),
    'max_episodes': 0,
    'lifespan': 0,
    'gamma': 0.9
}

trainer = Trainer(cfg)
trainer.policy_iteration(theta=1e-6)

print(trainer.agent.V)
# print(np.argmax(trainer.agent.pi, axis=-1))
print(trainer.world.occupancy_map)

Error = 2.165066207096613
Error = 1.3938365977135103
Error = 1.132117256786799
Error = 0.9726170421571334
Error = 0.8475901025529984
Error = 0.7544382336737163
Error = 0.6597661003771726
Error = 0.568193030627878
Error = 0.48348012812369046
Error = 0.40768132942586544
Error = 0.3414490263740557
Error = 0.28454084751736985
Error = 0.23622807672845703
Error = 0.195567029921472
Error = 0.16156227056641903
Error = 0.13325697279310234
Error = 0.10977775450148819
Error = 0.09035245340570164
Error = 0.07431259439886873
Error = 0.06108775321861337
Error = 0.050196107986934635
Error = 0.04123365488546327
Error = 0.03386345555091985
Error = 0.027805619090411682
Error = 0.022828331742152486
Error = 0.018740025384929027
Error = 0.015382655758690689
Error = 0.012626001239476992
Error = 0.010362868066858155
Error = 0.008505082702978228
Error = 0.00698015728815804
Error = 0.0057285244992559825
Error = 0.00470125037036162
Error = 0.0038581460546538437
Error = 0.0031662111809236393
Error = 0.0025983519